In [54]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import regex as re
import os
import seaborn as sns
from openpyxl import load_workbook
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
%reload_ext rpy2.ipython

In [55]:
os.chdir('/Users/arvind/Documents/Work/Zoho WorkDrive (Educational Initiatives Private Limited)/My Folders/Projects/CSF/Analysis/Code/DQA/')

<h1>
    Import raw data files
</h1>

In [43]:
#Importing all scored files
mp_literacy_scored = pd.read_excel("./2022_11_01_mp_literacy_total_scores.xlsx", index_col=[0])
mp_numeracy_scored = pd.read_excel("./2022_11_01_mp_numeracy_total_scores.xlsx", index_col=[0])
up_literacy_scored = pd.read_excel("./2022_11_01_up_literacy_total_scores.xlsx", index_col=[0])
up_numeracy_scored = pd.read_excel("./2022_11_01_up_numeracy_total_scores.xlsx", index_col=[0])

In [44]:
all(col in up_literacy_scored.columns for col in mp_literacy_scored.columns)
all(col in up_numeracy_scored.columns for col in mp_numeracy_scored.columns)

True

In [45]:
#Merging state files
literacy_scored = pd.concat([mp_literacy_scored,up_literacy_scored],join='inner')
numeracy_scored = pd.concat([mp_numeracy_scored,up_numeracy_scored],join='inner')

In [46]:
literacy_scored['school_details.State_label'].unique()

array(['Madhya Pradesh', 'Madhya_Pradesh', 'Uttar Pradesh',
       'Uttar_Pradesh'], dtype=object)

In [47]:
literacy_scored['school_details.State_label'] = literacy_scored['school_details.State_label'].replace('Madhya_Pradesh','Madhya Pradesh')
literacy_scored['school_details.State_label'] = literacy_scored['school_details.State_label'].replace('Uttar_Pradesh','Uttar Pradesh')
numeracy_scored['school_details.State_label'] = numeracy_scored['school_details.State_label'].replace('Madhya_Pradesh','Madhya Pradesh')
numeracy_scored['school_details.State_label'] = numeracy_scored['school_details.State_label'].replace('Uttar_Pradesh','Uttar Pradesh')
                                                                                                    

In [48]:
#Output for Tableau Workbook
literacy_scored.to_excel("../Results/literacy_scored.xlsx")
numeracy_scored.to_excel("../Results/numeracy_scored.xlsx")

In [65]:
numeracy_scored.columns.to_list()

['tabletUserName',
 'assessment_date',
 'school_details.State_label',
 'school_details.District_label',
 'school_details.Block_label',
 'school_details.School_label',
 'school_details.UDISE_cd_label',
 'SI_std_name',
 'student_age',
 'student_gender',
 'numeracy1_total',
 'numeracy1_tt_time_taken',
 'numeracy1_tt_cpm',
 'numeracy2_ut_total',
 'numeracy2_tt_total',
 'numeracy2_tt_time_taken',
 'numeracy2_tt_cpm',
 'numeracy3_total',
 'numeracy4_total',
 'numeracy5_total',
 'numeracy6_total',
 'numeracy7_total',
 'numeracy8_total',
 'numeracy9a_total',
 'numeracy9b_total']

In [73]:
#renaming variables for ease and use in R
literacy_scored = literacy_scored.rename(columns = {'school_details.State_label':'State',
                                  'school_details.District_label':'District',
                                  'school_details.Block_label':'Block',
                                  'school_details.School_label':'School',
                                  'school_details.UDISE_cd_label':'UDISE',
                                  'SI_std_name':'Name',
                                  'student_age':'Age',
                                  'student_gender':'Gender'})
numeracy_scored = numeracy_scored.rename(columns = {'school_details.State_label':'State',
                                  'school_details.District_label':'District',
                                  'school_details.Block_label':'Block',
                                  'school_details.School_label':'School',
                                  'school_details.UDISE_cd_label':'UDISE',
                                  'SI_std_name':'Name',
                                  'student_age':'Age',
                                  'student_gender':'Gender'})

In [76]:
literacy_scored.head()

,tabletUserName,assessment_date,State,District,Block,School,UDISE,Name,Age,Gender,literacy1_total,literacy2_total,literacy3_total,literacy4_ut_total,literacy4_tt_total,literacy4_tt_time_taken,literacy4_tt_cpm,literacy5_ut_total,literacy5_tt_total,literacy5_tt_time_taken,literacy5_tt_cpm,literacy6_total,literacy6_tt_time_taken,literacy6_tt_cpm,literacy7_total,literacy7_tt_time_taken,literacy7_tt_cpm,literacy8_reading_total,literacy8_comprehension_total,literacy9a_total,literacy9b_total,id
0,Jeevan,2022-09-17,Madhya Pradesh,Shajapur,M. Barodiya,GIRLS P.S. PALSAWAD,23220413303,Bhumika,5.0,Female,3,9,0,0,3,53,3,0,0,0,0,0,0,0,0,22,0,0,0,0,0,0
1,Kajal,2022-09-23,Madhya Pradesh,Vidisha,Gyaraspur,PS NIPANIYA,23310211501,pavan kushwah,6.0,Male,4,8,7,11,28,60,28,4,7,41,10,2,20,6,2,20,6,1,0,4,0,1
2,Rahnuma,2022-09-16,Madhya Pradesh,Shajapur,Shajapur,P.S. RATANPURA,23220603401,lakshmi gurjar,5.0,Female,1,7,0,0,0,17,0,0,0,0,0,0,0,0,0,18,0,0,0,10,10,2
3,Vaibhav,2022-09-23,Madhya Pradesh,Shajapur,Shajapur,BOYS M.S. MAHUPURA SHAJAPUR,23220605849,radhika rodu,6.0,Female,0,0,0,0,0,17,0,0,0,13,0,0,18,0,0,13,0,0,0,10,10,3
4,Nanda,2022-09-16,Madhya Pradesh,Shajapur,Shajapur,NEW M.S. DILLOD,23220619202,Subham Ramesh,6.0,Male,0,7,0,0,0,15,0,0,0,0,0,0,0,0,0,15,0,0,0,0,0,4


In [75]:
literacy_scored['id'] = literacy_scored.index
numeracy_scored['id'] = numeracy_scored.index

In [78]:
%%R -i literacy_scored

require(dplyr)
require(Hmisc)
require(sqldf)
sqldf("select State, Gender, count(distinct id) as ct from literacy_scored group by State, Gender")

           State Gender   ct
1 Madhya Pradesh Female 1096
2 Madhya Pradesh   Male  362
3  Uttar Pradesh Female 2788
4  Uttar Pradesh   Male  505


In [81]:
literacy_scored.groupby(['State','Gender'])['id'].count()

State           Gender
Madhya Pradesh  Female    1096
                Male       362
Uttar Pradesh   Female    2788
                Male       505
Name: id, dtype: int64

In [ ]:
literacy_scored[literacy_scored['']]